## Task 2: Basic ML on MNIST and FashionMNIST

### Task 2.1 One vs. all (rest) Linear and Logistic

### Task 2.2 One vs. One Linear and Logistic

### Task 2.3 Multi-class classifier (logistic softmax)

### Task 2.4 Comparison of results and discussion